In [1]:
import pandas as pd
import numpy as np
import sys
from pyspark.sql import functions as f
import re
from functools import reduce
from pyspark.sql import SparkSession
from typing import Dict
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, FloatType
from pandas.tseries.offsets import MonthEnd
from datetime import datetime
import re
from pyspark.sql.window import Window
import plotly.express as px
from pyspark.ml.feature import Bucketizer
from itertools import chain
from pandas.tseries.offsets import MonthEnd
from geopy import distance
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_columns', 200)

In [2]:
#!pip install scikit-learn --force
#!pip install catboost
#!pip install ipywidgets --force

In [3]:
df = pd.read_parquet('prepared_df.parquet')

In [4]:
df

,objectid,report_date,escrow_inflow,S_inflow,S_object,object_open_date,object_close_date,escrow_plan_close,escrow_fact_close,escrow_object_open_date,escrow_status,region_num,mean_price_per_metr,latitude,longitude,escrow_volume,S_volume,escrow_max_plan_close,escrow_max_fact_close,escrow_object_min_open_date,open_date,close_date,escrow_min_open_dt,escrow_max_open_dt,S_escrow/S_object,S_escrow,S_share,S
183,27214.0,2024-03-31,NaN,NaN,47367.00,2018-06-28,2024-03-31,NaT,NaT,NaT,Закрыт,77,NaN,55.881705,37.635428,NaN,NaN,2024-10-01,2023-12-18,2018-06-28,2018-06-28,2024-03-31,2021-07-31,2023-12-31,1.152980,54613.20,NaN,54613.20
498,34508.0,2024-03-31,NaN,NaN,65329.75,2019-09-23,2024-06-30,NaT,NaT,NaT,Закрыт,23,NaN,44.750342,37.730275,NaN,NaN,2024-12-31,2023-07-07,2019-09-23,2019-09-23,2024-06-30,2020-03-31,2024-02-29,0.888314,58033.36,NaN,58033.36
499,34508.0,2024-04-30,NaN,NaN,65329.75,2019-09-23,2024-06-30,NaT,NaT,NaT,Закрыт,23,NaN,44.750342,37.730275,NaN,NaN,2024-12-31,2023-07-07,2019-09-23,2019-09-23,2024-06-30,2020-03-31,2024-02-29,0.888314,58033.36,NaN,58033.36
500,34508.0,2024-05-31,NaN,NaN,65329.75,2019-09-23,2024-06-30,NaT,NaT,NaT,Закрыт,23,NaN,44.750342,37.730275,NaN,NaN,2024-12-31,2023-07-07,2019-09-23,2019-09-23,2024-06-30,2020-03-31,2024-02-29,0.888314,58033.36,NaN,58033.36
501,34508.0,2024-06-30,NaN,NaN,65329.75,2019-09-23,2024-06-30,NaT,NaT,NaT,Закрыт,23,NaN,44.750342,37.730275,NaN,NaN,2024-12-31,2023-07-07,2019-09-23,2019-09-23,2024-06-30,2020-03-31,2024-02-29,0.888314,58033.36,NaN,58033.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33715,46175.0,2023-10-31,1.821272e+08,1348.0,28002.40,2022-04-22,2027-06-30,2026-06-30,NaT,2022-04-22,Открыт,10,135109.202692,61.821115,34.311713,1.600238e+09,13265.1,2026-09-30,2024-01-31,2022-04-22,2022-04-22,2027-06-30,2022-08-31,2024-02-29,0.555163,15545.90,0.473713,28002.40
33716,46175.0,2023-11-30,7.547000e+07,542.6,28002.40,2022-04-22,2027-06-30,2026-06-30,NaT,2022-04-22,Открыт,10,139089.568743,61.821115,34.311713,1.675708e+09,13807.7,2026-09-30,2024-01-31,2022-04-22,2022-04-22,2027-06-30,2022-08-31,2024-02-29,0.555163,15545.90,0.493090,28002.40
33717,46175.0,2023-12-31,1.384829e+08,1029.5,28002.40,2022-04-22,2027-06-30,2026-06-30,NaT,2022-04-22,Открыт,10,134514.700340,61.821115,34.311713,1.814191e+09,14837.2,2026-09-30,2024-01-31,2022-04-22,2022-04-22,2027-06-30,2022-08-31,2024-02-29,0.555163,15545.90,0.529855,28002.40
33718,46175.0,2024-01-31,6.193682e+07,452.6,28002.40,2022-04-22,2027-06-30,2026-09-30,NaT,2022-04-22,Открыт,10,136846.698607,61.821115,34.311713,1.876128e+09,15289.8,2026-09-30,2024-01-31,2022-04-22,2022-04-22,2027-06-30,2022-08-31,2024-02-29,0.555163,15545.90,0.546017,28002.40


In [5]:
obj_df = df[df['report_date'] > pd.to_datetime('2019-01-01')].groupby('objectid').apply(lambda df: df.merge(pd.DataFrame(
                {'report_date': pd.date_range(df['report_date'].min() + MonthEnd(-3), df['report_date'].max(), freq = 'M')})).fillna(
                            {col: df[col].mode()[0] if len(df[col].mode()) > 0 else np.nan for col in ['latitude', 'longitude']})).reset_index(drop = True)


obj_df = obj_df.assign(year = obj_df['report_date'].map(lambda dt: dt.year), 
                       quartal = obj_df['report_date'].map(lambda dt: dt.quarter))[obj_df['report_date'].between(pd.to_datetime('2019-01-01'), pd.to_datetime('2024-03-01'))]

df_price = obj_df[[col for col in obj_df.columns if col not in ['latitude', 'longitude']]]

obj_df = obj_df[['year', 'quartal', 'objectid', 'latitude', 'longitude']].drop_duplicates(subset = ['year', 'quartal', 'objectid'])

In [6]:
df_price

,objectid,report_date,escrow_inflow,S_inflow,S_object,object_open_date,object_close_date,escrow_plan_close,escrow_fact_close,escrow_object_open_date,escrow_status,region_num,mean_price_per_metr,escrow_volume,S_volume,escrow_max_plan_close,escrow_max_fact_close,escrow_object_min_open_date,open_date,close_date,escrow_min_open_dt,escrow_max_open_dt,S_escrow/S_object,S_escrow,S_share,S,year,quartal
0,1802.0,2019-01-31,NaN,NaN,6247.72,2017-05-12,2021-12-31,NaT,NaT,NaT,Закрыт,18,NaN,0.0,0.00,2022-06-30,2022-01-14,2017-05-12,2017-05-12,2021-12-31,2020-10-31,2021-12-31,1.001609,6257.77,0.000000,6257.77,2019,1
1,1802.0,2019-02-28,NaN,NaN,6247.72,2017-05-12,2021-12-31,NaT,NaT,NaT,Закрыт,18,NaN,0.0,0.00,2022-06-30,2022-01-14,2017-05-12,2017-05-12,2021-12-31,2020-10-31,2021-12-31,1.001609,6257.77,0.000000,6257.77,2019,1
2,1802.0,2019-03-31,NaN,NaN,6247.72,2017-05-12,2021-12-31,NaT,NaT,NaT,Закрыт,18,NaN,0.0,0.00,2022-06-30,2022-01-14,2017-05-12,2017-05-12,2021-12-31,2020-10-31,2021-12-31,1.001609,6257.77,0.000000,6257.77,2019,1
3,1802.0,2019-04-30,NaN,NaN,6247.72,2017-05-12,2021-12-31,NaT,NaT,NaT,Закрыт,18,NaN,0.0,0.00,2022-06-30,2022-01-14,2017-05-12,2017-05-12,2021-12-31,2020-10-31,2021-12-31,1.001609,6257.77,0.000000,6257.77,2019,2
4,1802.0,2019-05-31,NaN,NaN,6247.72,2017-05-12,2021-12-31,NaT,NaT,NaT,Закрыт,18,NaN,0.0,0.00,2022-06-30,2022-01-14,2017-05-12,2017-05-12,2021-12-31,2020-10-31,2021-12-31,1.001609,6257.77,0.000000,6257.77,2019,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22764,46217.0,2023-10-31,29276920.0,359.99,7900.91,2021-12-28,2023-12-31,2024-06-30,2023-12-28,2021-12-28,Закрыт,16,81327.036862,256421370.0,3258.48,2024-07-01,2024-02-19,2021-12-28,2021-12-28,2024-02-29,2022-12-31,2024-02-29,0.679057,5365.17,0.607340,5365.17,2023,4
22765,46217.0,2023-11-30,67190650.0,805.50,7900.91,2021-12-28,2023-12-31,2024-06-30,2023-12-28,2021-12-28,Закрыт,16,83414.835506,323612020.0,4063.98,2024-07-01,2024-02-19,2021-12-28,2021-12-28,2024-02-29,2022-12-31,2024-02-29,0.679057,5365.17,0.757475,5365.17,2023,4
22766,46217.0,2023-12-31,106550848.0,1262.69,7900.91,2021-12-28,2023-12-31,2024-06-30,2024-02-19,2021-12-28,Закрыт,16,84384.011911,430162868.0,5326.67,2024-07-01,2024-02-19,2021-12-28,2021-12-28,2024-02-29,2022-12-31,2024-02-29,0.679057,5365.17,0.992824,5365.17,2023,4
22767,46217.0,2024-01-31,0.0,0.00,7900.91,2021-12-28,2023-12-31,NaT,NaT,2021-12-28,Закрыт,16,75341.345463,430162868.0,5326.67,2024-07-01,2024-02-19,2021-12-28,2021-12-28,2024-02-29,2022-12-31,2024-02-29,0.679057,5365.17,0.992824,5365.17,2024,1


In [7]:
obj_df.groupby('objectid').agg({'latitude': 'count', 'longitude': 'count'}).query('latitude != longitude')

,latitude,longitude
objectid,,


In [8]:
t = pd.concat([obj_df.sample(1)[['latitude','longitude']].reset_index(drop = True), 
           obj_df.sample(1)[['latitude','longitude']].rename(columns = dict(zip(['latitude','longitude'], ['latitude_part','longtitude_part']))).reset_index(drop = True)], axis = 1)
t

,latitude,longitude,latitude_part,longtitude_part
0,55.750337,37.70674,54.938751,20.146135


In [8]:
spark.table('prod_repl_cld_cian.cian_primary_premise').limit(100).toPandas()

,cian_id,realty_id,status_id,category_id,url,price_sqm,bargainterms_price,bargainterms_currency,creationdate,edit_date,extract_date,upload_date,exposure,views,contracttype_id,lat,lng,geo_userinput,geo_flag,geo_jk_name,geo_jk_id,geo_jk_house_name,geo_jk_house_id,roomscount,totalarea,minarea,livingarea,kitchenarea,floornumber,jk_house_flat_sectionnumber,jk_house_flat_flatnumber,jk_house_flat_flattype,decoration_id,conditiontype_id,repairtype_id,combinedwcscount,separatewcscount,balconiescount,loggiascount,parkingtype_id,roomsarea,source_id,bargainterms_clientfee,bargainterms_deposit,bargainterms_hasgraceperiod,bargainterms_vatincluded,isbyhomeowner,included_utility_charges,included_operational_costs,hasfurniture,undergrounds_name,undergrounds_time,undergrounds_transporttype,isapartments,ispenthouse,user_id,placementtype_id,inputtype_id,building_name,building_floorscount,buildingtype_id,buildingmaterialtype_id,acv_geo_region,acv_geo_raion_regiona,acv_geo_gorod,acv_geo_street,acv_geo_house,acv_geo_gorod_raw,acv_geo_dis_okrug,acv_geo_dis_raion,acv_geo_dis_mikroraion,korp_name,walls_material_id,korp_deadline_planned,korp_deadline_actual,korp_deadline_change,korp_stage_id,korp_finishing,has_fine_finishing,has_rough_finishing,is_without_finishing,korp_min_floor,korp_max_floor,cnt_studio,cnt_one_room_flat,cnt_two_room_flat,cnt_three_room_flat,cnt_four_room_flat,cnt_sv_pl_flat,cargo_lifts_cnt,passenger_lifts_cnt,undeground_parking_cnt,standalone_parking_cnt,guest_parking_cnt,cadastral_num,ceiling_height,website_jk,jk_name,developer,jk_alt_name,korp_class_id,building_id,gkh_series_id,building_ceiling_h,address,b_lat,b_lng,building_walls_type_id,building_floors_max,building_year_release,building_lifts,building_lifts_pass,building_lifts_service,building_renov,building_renov_refuse,building_energy_type_id,building_overlap_type_id,building_gas_type_id,building_chute_type_id,is_residence,bargainterms_prepay_months,bargainterms_pricetype_id,bargainterms_paymentperiod_id,classtype_id,system_dttm,datep
0,208608187,2005122.0,3,3,https://www.cian.ru/sale/flat/208608187,350000.0,42000000,rur,2019-05-28 00:00:00,2019-05-28 17:43:52,2020-05-19 00:00:00,2020-05-20 00:00:00,0,NaN,NaN,53.33294,83.69079,"Россия\, Алтайский край\, Барнаул\, Балтийская...",1,Test JK,270952.0,Test JK-House-1,1043178.0,2.0,120.00,NaN,NaN,NaN,2,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,2,NaN,NaN,None,None,None,False,False,None,None,NaN,None,None,None,28934895,NaN,NaN,None,13,NaN,NaN,Алтайский край,None,Барнаул,Балтийская улица,3,Барнаул,None,Индустриальный,None,None,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,1527714.0,NaN,NaN,"Россия\, Алтайский край\, Барнаул\, Балтийская...",53.3329,83.6908,3.0,10.0,2010.0,4.0,4.0,NaN,NaN,NaN,7.0,2.0,3.0,2.0,1.0,NaN,NaN,NaN,None,2022-07-05 09:31:24,2019-06-30
1,204681205,NaN,3,3,https://www.cian.ru/sale/flat/204681205,37999.0,2321000,rur,2019-04-04 00:00:00,2019-04-04 00:00:00,2020-05-19 00:00:00,2020-05-20 00:00:00,0,236.0,NaN,56.64068,47.86418,"Республика Марий Эл\, Йошкар-Ола\, улица Анциф...",1,Парк Победы,57947.0,Дом 1,581236.0,2.0,61.08,NaN,27.2,9.5,10,None,NaN,None,2.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,None,1,NaN,NaN,None,None,None,False,False,None,None,NaN,None,False,False,13405260,NaN,NaN,None,10,NaN,3.0,Республика Марий Эл,None,Йошкар-Ола,улица Анциферова,Дом 1,Йошкар-Ола,None,None,None,Дом 1,1.0,2019-10-01,2019-10-01,['2019-10-01'],3,None,None,None,None,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:05:0302008:2102,NaN,http://doma-12.ru,Парк Победы,НАШ ДОМ,None,1.0,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,2022-07-05 09:31:24,2019-06-30
2,209459858,3480244.0,3,3,https://www.cian.ru/sale/flat/209459858,350000.0,42000000,rur,2019-06-07 00:00:00,2019-06-07 03:11:13,2020-05-19 00:00:00,2020-05-20 00:00:00,0,NaN,NaN,57.80556,28.34716,"Россия\, Псковская область\, Псков\, улица Мет...",1

In [8]:
def merge_area_price(esc, df, apply_udf = False):
    
    df = df.withColumn('quartal', f.quarter('creationdate')).withColumn('year', f.year('creationdate'))
    df = esc.join(df, on = ['year', 'quartal'], how = 'left')
    print(df.count())
    r_Earth = 6372795
    
    get_dist = f.udf(lambda c1, c2: distance.geodesic(c1, c2).m, FloatType())
    
    if apply_udf:    
        df = df.withColumn('dist', 
                    get_dist(f.array('latitude', 'longitude'), f.array('latitude_part', 'longtitude_part'))
                          )
    else:
        df = df.withColumn('dist', 
                r_Earth*2*f.asin( 
                    (f.sin( (f.radians('lat') - f.radians('latitude')) / 2 )**2 + \
                     f.cos(f.radians('lat'))*f.cos(f.radians('latitude')) * f.sin( (f.radians('lng') - f.radians('longitude')) / 2 )**2 )**(1/2)
                                )
                      )
        
    w  = Window.partitionBy('objectid', 'year', 'quartal').orderBy('dist')
    df = df.withColumn('dist_rank', f.dense_rank().over(w))
    df = df.filter(f.col('dist_rank') <= 70)
    return df
dfp = merge_area_price(spark.createDataFrame(obj_df), spark.table('prod_repl_cld_cian.cian_primary_premise')).toPandas()

9293847275


Py4JJavaError: An error occurred while calling o190.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 14.0 failed 4 times, most recent failure: Lost task 6.3 in stage 14.0 (TID 183) (p0dapp-ah2067xp.region.vtb.ru executor 11): ExecutorLostFailure (executor 11 exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 176618 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2672)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2608)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2607)
	at scala.collection.immutable.List.foreach(List.scala:333)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2607)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1182)
	at org.apache.spark.scheduler.DAGScheduler.$a

In [9]:
def merge_area_price(esc, df, apply_udf = False):
    df = esc.join(df, on = ['year', 'quartal'], how = 'left')
    print(df.count())
    r_Earth = 6372795
    
    get_dist = f.udf(lambda c1, c2: distance.geodesic(c1, c2).m, FloatType())
    
    if apply_udf:    
        df = df.withColumn('dist', 
                    get_dist(f.array('latitude', 'longitude'), f.array('latitude_part', 'longtitude_part'))
                          )
    else:
        df = df.withColumn('dist', 
                r_Earth*2*f.asin( 
                    (f.sin( (f.radians('latitude_part') - f.radians('latitude')) / 2 )**2 + \
                     f.cos(f.radians('latitude_part'))*f.cos(f.radians('latitude')) * f.sin( (f.radians('longtitude_part') - f.radians('longitude')) / 2 )**2 )**(1/2)
                                )
                      )
        
    w  = Window.partitionBy('objectid', 'year', 'quartal').orderBy('dist')
    df = df.withColumn('dist_rank', f.dense_rank().over(w))
    df = df.filter(f.col('dist_rank') <= 50)
    return df
dfp = merge_area_price(spark.createDataFrame(obj_df), spark.table('prod_dm_cian.cd_soba_primary')).toPandas()

237850553


/opt/cloudera/parcels/AnacondaPy37/envs/python3.7.9/lib/python3.7/site-packages/pyarrow/util.py:43: FutureWarning: pyarrow.open_stream is deprecated as of 0.17.0, please use pyarrow.ipc.open_stream instead.
  warnings.warn(msg, FutureWarning)
[Stage 5:==================================================>   (186 + 14) / 200]24/04/19 02:06:37 ERROR cluster.YarnScheduler: Lost executor 7 on p0drp0-db5051xp.region.vtb.ru: Unable to create executor due to Address already in use: Service 'org.apache.spark.network.netty.NettyBlockTransferService' failed after 3 retries (starting from 61765)! Consider explicitly setting the appropriate port for the service 'org.apache.spark.network.netty.NettyBlockTransferService' (for example spark.ui.port for SparkUI) to an available port or increasing spark.port.maxRetries.
24/04/19 02:06:37 ERROR cluster.YarnScheduler: Lost executor 11 on p0drp0-db5051xp.region.vtb.ru: Unable to create executor due to Address already in use: Service 'org.apache.spark.network

In [7]:
dfp.count()

year                        1743172
quartal                     1743172
objectid                    1743172
latitude                     615740
longitude                    615740
jk_id                       1742054
jk_nm                       1742026
latitude_part               1742054
longtitude_part             1742054
city                        1742054
street                      1719715
house                       1648463
district                    1422040
class_nm                    1031768
rooms_cnt                   1742054
area_range_part             1742054
flat_cnt                    1742054
total_area_part             1742054
offer_price_sqm_amt         1742054
offer_currency_price_nm     1742054
developer_nm                1210653
currency_price_nm           1031326
deadline_actual             1096081
deadline_change             1085959
deadline_plan               1154204
count_fl_cnt                   6477
count_ul_cnt                   6477
bought_price_sqm_info       

In [13]:
def filter_dist(df):
    
    def get_scaled_dist(df, col):
        q25 = df[col].quantile(0.25)
        q75 = df[col].quantile(0.75)
        IQR = q75 - q25
        std = df[col][df[col] <= q75 + 1.5*IQR].std()
        df = df.assign(dist_weight = 1 / ((df[col] - df[col].min())/std + 1))
        return df
    
    df = df.groupby(['objectid', 'year', 'quartal']).apply(lambda df: get_scaled_dist(df, 'dist')).reset_index(drop = True)
    df = df.groupby(['objectid', 'year', 'quartal'], as_index = False).apply(lambda df: (df['total_area_part'] * df['dist_weight'] * df['offer_price_sqm_amt']).sum() / (df['total_area_part'] * df['dist_weight']).sum())
    df.columns = ['objectid', 'year', 'quartal', 'mean_price']
    return df
    
fdf = filter_dist(dfp).merge(df_price, on = ['objectid', 'year', 'quartal'], how = 'outer')
#for _, df_ in list(filter_dist(dfp).merge(df_price, on = ['objectid', 'year', 'quartal'], how = 'outer').groupby(['objectid']))[:50]:
#    display(df_)
#    px.line(df_.sort_values('report_date'), x = 'report_date', y = ['mean_price', 'mean_price_per_metr']).show()
#    #px.line(df_.assign(ds = df_['year'].astype('str') + ' q' + df_['quartal'].astype('str')), x = 'ds', y = 'mean_price').show()

/opt/cloudera/parcels/AnacondaPy37/envs/python3.7.9/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning:

invalid value encountered in double_scalars



In [15]:
fdf

,objectid,year,quartal,mean_price,report_date,escrow_inflow,S_inflow,S_object,object_open_date,object_close_date,escrow_plan_close,escrow_fact_close,escrow_object_open_date,escrow_status,region_num,mean_price_per_metr,escrow_volume,S_volume,escrow_max_plan_close,escrow_max_fact_close,escrow_object_min_open_date,open_date,close_date,escrow_min_open_dt,escrow_max_open_dt,S_escrow/S_object,S_escrow,S_share,S
0,1802.0,2019,1,49766.597215,2019-01-31,NaN,NaN,6247.72,2017-05-12,2021-12-31,NaT,NaT,NaT,Закрыт,18,NaN,0.0,0.00,2022-06-30,2022-01-14,2017-05-12,2017-05-12,2021-12-31,2020-10-31,2021-12-31,1.001609,6257.77,0.000000,6257.77
1,1802.0,2019,1,49766.597215,2019-02-28,NaN,NaN,6247.72,2017-05-12,2021-12-31,NaT,NaT,NaT,Закрыт,18,NaN,0.0,0.00,2022-06-30,2022-01-14,2017-05-12,2017-05-12,2021-12-31,2020-10-31,2021-12-31,1.001609,6257.77,0.000000,6257.77
2,1802.0,2019,1,49766.597215,2019-03-31,NaN,NaN,6247.72,2017-05-12,2021-12-31,NaT,NaT,NaT,Закрыт,18,NaN,0.0,0.00,2022-06-30,2022-01-14,2017-05-12,2017-05-12,2021-12-31,2020-10-31,2021-12-31,1.001609,6257.77,0.000000,6257.77
3,1802.0,2019,2,51191.024998,2019-04-30,NaN,NaN,6247.72,2017-05-12,2021-12-31,NaT,NaT,NaT,Закрыт,18,NaN,0.0,0.00,2022-06-30,2022-01-14,2017-05-12,2017-05-12,2021-12-31,2020-10-31,2021-12-31,1.001609,6257.77,0.000000,6257.77
4,1802.0,2019,2,51191.024998,2019-05-31,NaN,NaN,6247.72,2017-05-12,2021-12-31,NaT,NaT,NaT,Закрыт,18,NaN,0.0,0.00,2022-06-30,2022-01-14,2017-05-12,2017-05-12,2021-12-31,2020-10-31,2021-12-31,1.001609,6257.77,0.000000,6257.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21158,46217.0,2022,3,NaN,2022-08-31,NaN,NaN,7900.91,2021-12-28,2023-12-31,NaT,NaT,NaT,Закрыт,16,NaN,0.0,0.00,2024-07-01,2024-02-19,2021-12-28,2021-12-28,2024-02-29,2022-12-31,2024-02-29,0.679057,5365.17,0.000000,5365.17
21159,46217.0,2022,3,NaN,2022-09-30,NaN,NaN,7900.91,2021-12-28,2023-12-31,NaT,NaT,NaT,Закрыт,16,NaN,0.0,0.00,2024-07-01,2024-02-19,2021-12-28,2021-12-28,2024-02-29,2022-12-31,2024-02-29,0.679057,5365.17,0.000000,5365.17
21160,46217.0,2023,2,NaN,2023-04-30,25872200.0,324.55,7900.91,2021-12-28,2023-12-31,2024-07-01,2023-12-28,2021-12-28,Закрыт,16,79500.000000,60657040.0,777.14,2024-07-01,2024-02-19,2021-12-28,2021-12-28,2024-02-29,2022-12-31,2024-02-29,0.679057,5365.17,0.144849,5365.17
21161,46217.0,2023,2,NaN,2023-05-31,16595680.0,218.24,7900.91,2021-12-28,2023-12-31,2024-07-01,2023-12-28,2021-12-28,Закрыт,16,77750.000000,77252720.0,995.38,2024-07-01,2024-02-19,2021-12-28,2021-12-28,2024-02-29,2022-12-31,2024-02-29,0.679057,5365.17,0.185526,5365.17


24/04/19 10:04:20 WARN security.UserGroupInformation: Exception encountered while running the renewal command for vtb70175290@REGION.VTB.RU. (TGT end time:1713517460000, renewalFailures: 0, renewalFailuresTotal: 1)
ExitCodeException exitCode=1: kinit: Bad format in credentials cache (filename: /home/vtb70175290/.krb5cc_1000650088) while storing credentials

	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1009)
	at org.apache.hadoop.util.Shell.run(Shell.java:902)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1227)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1321)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1303)
	at org.apache.hadoop.security.UserGroupInformation$TicketCacheRenewalRunnable.relogin(UserGroupInformation.java:1077)
	at org.apache.hadoop.security.UserGroupInformation$AutoRenewalForUserCredsRunnable.run(UserGroupInformation.java:988)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.

In [14]:
fdf.to_parquet('price_df.parquet')

In [7]:
fdf = pd.read_parquet('price_df.parquet') 

In [12]:
fdf[['objectid', 'report_date', *[col for col in fdf.columns if col not in df_price.columns]]].merge(df_price, on = ['objectid', 'report_date']).to_parquet('price_df.parquet')

24/04/25 09:08:41 WARN security.UserGroupInformation: Exception encountered while running the renewal command for vtb70175290@REGION.VTB.RU. (TGT end time:1714032521000, renewalFailures: 0, renewalFailuresTotal: 1)
ExitCodeException exitCode=1: kinit: Bad format in credentials cache (filename: /home/vtb70175290/.krb5cc_1000650088) while storing credentials

	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1009)
	at org.apache.hadoop.util.Shell.run(Shell.java:902)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1227)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1321)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1303)
	at org.apache.hadoop.security.UserGroupInformation$TicketCacheRenewalRunnable.relogin(UserGroupInformation.java:1077)
	at org.apache.hadoop.security.UserGroupInformation$AutoRenewalForUserCredsRunnable.run(UserGroupInformation.java:988)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.

In [38]:
for _, df_ in list(dfp.groupby(['objectid']))[:60]:
    pass
    #display(df_)
    #px.histogram(df_, x = 'dist').show()
    #px.box(df_.assign(ds = df_['year'].astype('str') + ' q' + df_['quartal'].astype('str')), x = 'dist', color = 'ds').show()